In [2]:
import os 

from tensorflow import keras
import tensorflow as tf
import numpy as np
from PIL import Image
from skimage.color import rgb2ycbcr, ycbcr2rgb


from FSRCNN.model import FSRCNN

In [3]:
# load model 
IS_FSRCNN_S = True
name_model = f"FSRCNN_S" if IS_FSRCNN_S else "FSRCNN"
checkpoint_path  = f"checkpoints/"+ name_model+"{epoch:03d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
os.listdir(checkpoint_dir)
latest = tf.train.latest_checkpoint(checkpoint_dir)
model = FSRCNN(d=32, s=5, m=1, r=4) if IS_FSRCNN_S else FSRCNN()
model.load_weights(latest)


data_dir = 'data/DIV2K/DIV2K_train_HR/'
files = os.listdir(data_dir)



def eval_image(img, upscaler, r ): 
    '''
    Args: 
     - img: PIL image in RGB
     - upscaler: 
     - r: 
    '''

def eval_dataset(img_dir, upscalers, output_dir):
    '''
    Function: 
        Save image stacks of lr, hr, and sr of an image upscaled by methods in upscalers into output_dir 
    Args: 
        img_dir: the img to stack 
        upscalers: a list of upscaling methods 
        output_dir: the output directory 
    '''
    n = len(upscalers) # two for bicubic and original image
    hr = Image.open(img_dir)
    # crop image size to the max multiple 4
    hr = hr.crop( (0,0, hr.size[0]//4 *4, hr.size[1]//4*4  )   ) 
    lr = hr.resize((hr.size[0]//4, hr.size[1]//4), Image.BICUBIC)
    bi = lr.resize(hr.size, Image.BICUBIC)
    dst = Image.new('RGB', (hr.width*(n+2), hr.height))
    dst.paste(bi, (0, 0))
    for i in range(n):
        upscaler = upscalers[i]
        sr = upscaler(lr)
        dst.paste(sr, (hr.width*(i+1), 0))
        
        np_sr = postprocess(sr)
        np_hr = postprocess(hr)
        psnr = peak_signal_noise_ratio(np_hr,np_sr,data_range=255)
        ssim = structural_similarity(np_hr, np_sr, data_range=255)
        print("%.2f / %.2f" % (psnr, ssim))

    dst.paste(hr, (hr.width*(n+1), 0))
    dst.save(output_dir)
    print("Image saved at: " + output_dir)

for f in files: 
    # load image from path
    hr = Image.open(data_dir + f)
    hr = hr.crop( (0,0, hr.size[0]//4 *4, hr.size[1]//4*4  )   ) 
    lr = hr.resize((hr.size[0]//4, hr.size[1]//4), Image.BICUBIC)

    
    yuv = (rgb2ycbcr(lr) - 16 )/ 219
    y = yuv[...,0]
    y = y[None,...,None]
    y = tf.cast(y, tf.float32)
    # the model only deals with y-channel
    y_sr = model(y) 
    
    print(hr.size)
    print(lr.size)
    print(y_sr.shape)

    # stich y and uv channels together in the hr space
    w, h = yuv.shape[0], yuv.shape[1]
    hr = tf.image.resize(yuv, size=[w*4,h*4], method='bicubic', antialias=True) 

    print(hr.shape)
    hr = tf.concat( (y_sr, hr[...,1][None,...,None], hr[...,2][None,...,None]  ) , axis= -1)[0].numpy()* 219 + 16
    hr = np.clip(  hr, 16, 235 )

    # convert back to rgb
    rgb = np.clip( ycbcr2rgb(hr)*255, 0, 255).astype(np.uint8)
    Image.fromarray(rgb).show()

    bi = lr.resize( (lr.size[0]*4, lr.size[1]*4), Image.BICUBIC)
    bi.show()

    break

   



FileNotFoundError: [WinError 3] The system cannot find the path specified: 'checkpoints'